In [1]:
# bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from bs4 import BeautifulSoup
import re
import csv
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import hamming_loss
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB
from skmultilearn.problem_transform import ClassifierChain
from sklearn.linear_model import LogisticRegression
from skmultilearn.problem_transform import LabelPowerset
from skmultilearn.adapt import MLkNN
from sklearn.multiclass import OneVsRestClassifier
from scipy.sparse import csr_matrix, lil_matrix
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, recall_score, f1_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn import neighbors
from sklearn import metrics
from nltk.tokenize import sent_tokenize
from pandas.api.types import CategoricalDtype
import pickle

In [2]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('portuguese'))

def clean_text(text):
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text

stemmer = SnowballStemmer("portuguese")

def stemming(sentence):
    stemSentence = ""
    for word in sentence.split():
        stem = stemmer.stem(word)
        stemSentence += stem
        stemSentence += " "
    stemSentence = stemSentence.strip()
    return stemSentence

In [3]:
df = pd.read_csv('C:/Users/Márcio/Downloads/PEDRO/Projeto/teste12.csv', encoding = 'utf=8')
df.columns = ["Index", "Título", "Classes", "Link", "Conteúdo"]
df

,Index,Título,Classes,Link,Conteúdo
0,1,Táxis tailandeses viram hortas após queda na d...,"['Agro', ' Tailândia ']",https://g1.globo.com/economia/agronegocios/not...,Taxistas tailandeses decidiram transformar se...
1,2,Agricultores competem pelo título de 'tomate m...,"['Agro', ' Espanha ']",https://g1.globo.com/economia/agronegocios/not...,Dezenas de agricultores participaram do concu...
2,3,De onde vem o que eu como: aplicativos conecta...,"['Agro', ' CNA ', ' IBGE ', ' Mato Grosso...",https://g1.globo.com/economia/agronegocios/agr...,Aplicativos de compra e venda de comida não sã...
3,4,Agricultura familiar produz quase metade do mi...,['Agro'],https://g1.globo.com/economia/agronegocios/agr...,Uma boa quantidade dos alimentos que chegam na...
4,5,Cooperativas do MST atingem meta de captar R$ ...,"['Agro', ' MST ']",https://g1.globo.com/economia/agronegocios/not...,Sete cooperativas do Movimento dos Trabalhador...
5,6,Projeto faz mapeamento inédito de banco de ali...,['Agro'],https://g1.globo.com/economia/agronegocios/not...,Um projeto da Ação da Cidadania e do Google es...
6,7,Minas Gerais conquista 40 medalhas em concurso...,['Agro'],https://g1.globo.com/mg/minas-gerais/noticia/2...,O sabor de Minas Gerais foi destaque no Mondia...
7,8,Minifruta: agricultor cultiva maçãs do tamanho...,"['Agro', ' Rússia ']",https://g1.globo.com/economia/agronegocios/not...,"O produtor Nikifor Ivanov, de Yakutsk, na Rúss..."
8,9,"Pimentão, abobrinha e pepino: preços das horta...","['Agro', ' IBGE ']",https://g1.globo.com/economia/agronegocios/not...,Os preços hortaliças dispararam em agosto em r...
9,10,Expectativa de colheita de grãos tem mais uma ...,"['Agro', ' CONAB ']",https://g1.globo.com/economia/agronegocios/not...,A expectativa da colheita dos grãos no Brasil ...


In [5]:
def busca(palavra, lista):
    saida = False
    for word in lista:
        if word == palavra:
            saida = True
    return saida

def substituir(texto):
    for i in range(len(texto)):
        texto[i] = texto[i].replace("[", "")
        texto[i] = texto[i].replace("'", "")
        texto[i] = texto[i].replace("]", "")


classes_list = []
for i in df['Classes']:
    texto = i.split(",")
    substituir(texto)
    for j in texto:
        repeticao = busca(j, classes_list)
        if repeticao == False:
            classes_list.append(j)

classes_list2 = []
for i in classes_list:
    classes_list2.append(i.replace(' ', '')) 

classes_list2.sort()
classes_list2

['Agro',
 'Brasil',
 'CNA',
 'CNI',
 'CONAB',
 'Canadá',
 'Chile',
 'China',
 'Espanha',
 'EstadosUnidos',
 'Facebook',
 'IBGE',
 'Jordânia',
 'MST',
 'MatoGrosso',
 'Pará',
 'PortodeSantos',
 'Rússia',
 'SãoPaulo',
 'Tailândia',
 'Turquia',
 'UFRA',
 'UFRJ',
 'USP']

In [6]:
noticias =  df

noticias['Conteúdo'] = noticias['Conteúdo'].apply(clean_text)
noticias['Conteúdo'] = noticias['Conteúdo'].apply(stemming)
noticias['Título'] = noticias['Título'].apply(clean_text)
noticias['Título'] = noticias['Título'].apply(stemming)

#noticias['Classes'] = classes_list[:17]

noticias

,Index,Título,Classes,Link,Conteúdo
0,1,txis tailandes vir hort aps qued demand durant...,"['Agro', ' Tailândia ']",https://g1.globo.com/economia/agronegocios/not...,taxist tailandes decid transform carr pequen h...
1,2,agricultor compet ttul tomat fei espanh,"['Agro', ' Espanha ']",https://g1.globo.com/economia/agronegocios/not...,dezen agricultor particip concurs tomat fei es...
2,3,onde vem aplic conect consumidor aliment agric...,"['Agro', ' CNA ', ' IBGE ', ' Mato Grosso...",https://g1.globo.com/economia/agronegocios/agr...,aplic compr vend com so novidad trat comercial...
3,4,agricultur famili produz quas metad milh frang...,['Agro'],https://g1.globo.com/economia/agronegocios/agr...,boa quantidad aliment cheg mes brasileir vem f...
4,5,cooper mst ating met capt r 17 5 milh merc fin...,"['Agro', ' MST ']",https://g1.globo.com/economia/agronegocios/not...,set cooper moviment trabalh rur terr mst concl...
5,6,projet faz mapeament indit banc aliment cozinh...,['Agro'],https://g1.globo.com/economia/agronegocios/not...,projet cidadan googl est realiz mapeament indi...
6,7,min ger conquist 40 medalh concurs mundial que...,['Agro'],https://g1.globo.com/mg/minas-gerais/noticia/2...,sabor min ger destaqu mondial du fromag et des...
7,8,minifrut agricultor cultiv tamanh prol rssi,"['Agro', ' Rússia ']",https://g1.globo.com/economia/agronegocios/not...,produtor nikifor ivanov yakutsk rssi cultiv mi...
8,9,piment abobrinh pepin pre hortal disp agost gead,"['Agro', ' IBGE ']",https://g1.globo.com/economia/agronegocios/not...,pre hortal dispar agost rela julh aps efeit ge...
9,10,expect colheit gros qued problem clim diz conab,"['Agro', ' CONAB ']",https://g1.globo.com/economia/agronegocios/not...,expect colheit gros brasil cicl 2020 21 qued c...


In [8]:
def nova_coluna(lista_tags, dataframe, nome_tags):
    for tag in lista_tags: # compara com cada palavra da lista
        coluna = list() 
        for linha_tag in dataframe[nome_tags]:
            if tag in linha_tag:
                #tag = tag[1:-1]
                coluna.append(1)
            else:
                coluna.append(0)
        dataframe[tag] = coluna

nova_coluna(classes_list2,noticias,'Classes')
noticias

,Index,Título,Classes,Link,Conteúdo,Agro,Brasil,CNA,CNI,CONAB,...,MatoGrosso,Pará,PortodeSantos,Rússia,SãoPaulo,Tailândia,Turquia,UFRA,UFRJ,USP
0,1,txis tailandes vir hort aps qued demand durant...,"['Agro', ' Tailândia ']",https://g1.globo.com/economia/agronegocios/not...,taxist tailandes decid transform carr pequen h...,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,2,agricultor compet ttul tomat fei espanh,"['Agro', ' Espanha ']",https://g1.globo.com/economia/agronegocios/not...,dezen agricultor particip concurs tomat fei es...,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,onde vem aplic conect consumidor aliment agric...,"['Agro', ' CNA ', ' IBGE ', ' Mato Grosso...",https://g1.globo.com/economia/agronegocios/agr...,aplic compr vend com so novidad trat comercial...,1,0,1,0,0,...,0,1,0,0,0,0,0,1,1,1
3,4,agricultur famili produz quas metad milh frang...,['Agro'],https://g1.globo.com/economia/agronegocios/agr...,boa quantidad aliment cheg mes brasileir vem f...,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,cooper mst ating met capt r 17 5 milh merc fin...,"['Agro', ' MST ']",https://g1.globo.com/economia/agronegocios/not...,set cooper moviment trabalh rur terr mst concl...,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,6,projet faz mapeament indit banc aliment cozinh...,['Agro'],https://g1.globo.com/economia/agronegocios/not...,projet cidadan googl est realiz mapeament indi...,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,7,min ger conquist 40 medalh concurs mundial que...,['Agro'],https://g1.globo.com/mg/minas-gerais/noticia/2...,sabor min ger destaqu mondial du fromag et des...,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,8,minifrut agricultor cultiv tamanh prol rssi,"['Agro', ' Rússia ']",https://g1.globo.com/economia/agronegocios/not...,produtor nikifor ivanov yakutsk rssi cultiv mi...,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
8,9,piment abobrinh pepin pre hortal disp agost gead,"['Agro', ' IBGE ']",https://g1.globo.com/economia/agronegocios/not...,pre hortal dispar agost rela julh aps efeit ge...,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,10,expect colheit gros qued problem clim diz conab,"['Agro', ' CONAB ']",https://g1.globo.com/economia/agronegocios/not...,expect colheit gros brasil cicl 2020 21 qued c...,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [7]:
train, test = train_test_split(noticias, random_state=42, test_size=0.33, shuffle=True)

X_train = train.Conteúdo
X_test = test.Conteúdo
print(X_train.shape)
print(X_test.shape)

(11,)
(6,)


In [14]:
# Define a pipeline combining a text feature extractor with multi lable classifier
NB_pipeline = Pipeline([('tfidf', TfidfVectorizer()),('clf', OneVsRestClassifier(MultinomialNB(fit_prior=True, class_prior=None))),])

for category in classes_list2:
    print('... Processing {}'.format(category))
    # train the model using X_dtm & y
    NB_pipeline.fit(X_train, train[category])
    # compute the testing accuracy
    prediction = NB_pipeline.predict(X_test)
    print('Test accuracy is {}'.format(accuracy_score(test[category], prediction)))
    print('{}'.format(classification_report(test[category], prediction, zero_division=0)))

... Processing Agro
Test accuracy is 1.0
              precision    recall  f1-score   support

           1       1.00      1.00      1.00         6

    accuracy                           1.00         6
   macro avg       1.00      1.00      1.00         6
weighted avg       1.00      1.00      1.00         6

... Processing Brasil
Test accuracy is 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6

    accuracy                           1.00         6
   macro avg       1.00      1.00      1.00         6
weighted avg       1.00      1.00      1.00         6

... Processing CNA
Test accuracy is 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6

    accuracy                           1.00         6
   macro avg       1.00      1.00      1.00         6
weighted avg       1.00      1.00      1.00         6

... Processing CNI
Test accuracy is 1.0
              pre

C:\Users\Márcio\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\multiclass.py:80: UserWarning: Label not 1 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
C:\Users\Márcio\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\multiclass.py:80: UserWarning: Label not 0 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
C:\Users\Márcio\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\multiclass.py:80: UserWarning: Label not 0 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
C:\Users\Márcio\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\multiclass.py:80: UserWarning: Label not 0 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
C:\Users\Márcio\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\multiclass.


... Processing PortodeSantos
Test accuracy is 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6

    accuracy                           1.00         6
   macro avg       1.00      1.00      1.00         6
weighted avg       1.00      1.00      1.00         6

... Processing Rússia
Test accuracy is 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6

    accuracy                           1.00         6
   macro avg       1.00      1.00      1.00         6
weighted avg       1.00      1.00      1.00         6

... Processing SãoPaulo
Test accuracy is 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6

    accuracy                           1.00         6
   macro avg       1.00      1.00      1.00         6
weighted avg       1.00      1.00      1.00         6

... Processing Tailândia
Test accuracy is 

C:\Users\Márcio\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\multiclass.py:80: UserWarning: Label not 0 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
C:\Users\Márcio\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\multiclass.py:80: UserWarning: Label not 0 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
C:\Users\Márcio\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\multiclass.py:80: UserWarning: Label not 0 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %


In [13]:
# Define a pipeline combining a text feature extractor with multi lable classifier
SVC_pipeline = Pipeline([('tfidf', TfidfVectorizer()),('clf', OneVsRestClassifier(LinearSVC(), n_jobs=1)),])

for category in classes_list2:
    print('... Processing {}'.format(category))
    # train the model using X_dtm & y
    SVC_pipeline.fit(X_train, train[category])
    # compute the testing accuracy
    prediction = SVC_pipeline.predict(X_test)
    print('Test accuracy is {}'.format(accuracy_score(test[category], prediction)))
    print('{}'.format(classification_report(test[category], prediction, zero_division=0)))

... Processing Agro
Test accuracy is 1.0
              precision    recall  f1-score   support

           1       1.00      1.00      1.00         6

    accuracy                           1.00         6
   macro avg       1.00      1.00      1.00         6
weighted avg       1.00      1.00      1.00         6

... Processing Brasil
Test accuracy is 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6

    accuracy                           1.00         6
   macro avg       1.00      1.00      1.00         6
weighted avg       1.00      1.00      1.00         6

... Processing CNA
Test accuracy is 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6

    accuracy                           1.00         6
   macro avg       1.00      1.00      1.00         6
weighted avg       1.00      1.00      1.00         6

... Processing CNI
Test accuracy is 1.0
              pre

C:\Users\Márcio\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\multiclass.py:80: UserWarning: Label not 1 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
C:\Users\Márcio\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\multiclass.py:80: UserWarning: Label not 0 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
C:\Users\Márcio\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\multiclass.py:80: UserWarning: Label not 0 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
C:\Users\Márcio\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\multiclass.py:80: UserWarning: Label not 0 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
C:\Users\Márcio\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\multiclass.


... Processing SãoPaulo
Test accuracy is 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6

    accuracy                           1.00         6
   macro avg       1.00      1.00      1.00         6
weighted avg       1.00      1.00      1.00         6

... Processing Tailândia
Test accuracy is 0.8333333333333334
              precision    recall  f1-score   support

           0       0.83      1.00      0.91         5
           1       0.00      0.00      0.00         1

    accuracy                           0.83         6
   macro avg       0.42      0.50      0.45         6
weighted avg       0.69      0.83      0.76         6

... Processing Turquia
Test accuracy is 0.8333333333333334
              precision    recall  f1-score   support

           0       0.83      1.00      0.91         5
           1       0.00      0.00      0.00         1

    accuracy                           0.83         6
   macro avg    

C:\Users\Márcio\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\multiclass.py:80: UserWarning: Label not 0 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
C:\Users\Márcio\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\multiclass.py:80: UserWarning: Label not 0 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
C:\Users\Márcio\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\multiclass.py:80: UserWarning: Label not 0 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %


In [11]:
# Define a pipeline combining a text feature extractor with multi lable classifier
LogReg_pipeline = Pipeline([('tfidf', TfidfVectorizer()),('clf', OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=1)),])

for category in classes_list2:
    print('... Processing {}'.format(category))
    # train the model using X_dtm & y
    LogReg_pipeline.fit(X_train, train[category])
    # compute the testing accuracy
    prediction = LogReg_pipeline.predict(X_test)
    print('Test accuracy is {}'.format(accuracy_score(test[category], prediction)))
    print('{}'.format(classification_report(test[category], prediction, zero_division=0)))

... Processing Agro
Test accuracy is 1.0
              precision    recall  f1-score   support

           1       1.00      1.00      1.00         6

    accuracy                           1.00         6
   macro avg       1.00      1.00      1.00         6
weighted avg       1.00      1.00      1.00         6

... Processing Brasil
Test accuracy is 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6

    accuracy                           1.00         6
   macro avg       1.00      1.00      1.00         6
weighted avg       1.00      1.00      1.00         6

... Processing CNA
Test accuracy is 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6

    accuracy                           1.00         6
   macro avg       1.00      1.00      1.00         6
weighted avg       1.00      1.00      1.00         6

... Processing CNI
Test accuracy is 1.0
              pre

C:\Users\Márcio\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\multiclass.py:80: UserWarning: Label not 1 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
C:\Users\Márcio\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\Users\Márcio\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\Users\Márcio\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\Users\Márcio\AppData\Local\Pr

Test accuracy is 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6

    accuracy                           1.00         6
   macro avg       1.00      1.00      1.00         6
weighted avg       1.00      1.00      1.00         6

... Processing MatoGrosso
Test accuracy is 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6

    accuracy                           1.00         6
   macro avg       1.00      1.00      1.00         6
weighted avg       1.00      1.00      1.00         6

... Processing Pará
Test accuracy is 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6

    accuracy                           1.00         6
   macro avg       1.00      1.00      1.00         6
weighted avg       1.00      1.00      1.00         6

... Processing PortodeSantos
Test accuracy is 1.0
              precisio

C:\Users\Márcio\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\Users\Márcio\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\multiclass.py:80: UserWarning: Label not 0 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
C:\Users\Márcio\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\Users\Márcio\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\multiclass.py:80: UserWarning: Label not 0 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
C:\Users\Márcio\AppData\Local\Programs\Python\Python